In [44]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import RobustScaler,StandardScaler,MinMaxScaler
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
import plotly.express as px

In [45]:
df = pd.read_csv("sample_df.csv")
test = pd.read_csv("test.csv")
sub = pd.read_csv("sample_submission.csv")

In [46]:
useless_col = ["game_num","event_id","event_time","player_scoring_next","team_scoring_next","Unnamed: 0"]

In [47]:
df.drop(columns = useless_col,inplace = True)

In [48]:
#df.dropna(axis=0, inplace=True)  EKSİK OYUNCU SAYISI EKLENECEĞİ İÇİN BOŞ SATIRLAR SİLİNMİYOR

In [49]:
dff_a = df[["p0_pos_x","p1_pos_x","p2_pos_x"]]
dff_b = df[["p3_pos_x","p4_pos_x","p5_pos_x"]]

df["team_a_eksik"] = dff_a.isnull().sum(axis=1)
df["team_b_eksik"] = dff_b.isnull().sum(axis=1)

In [50]:
df['ball_distance_to_goal_A'] = np.sqrt((df["ball_pos_x"])**2 + (df["ball_pos_y"] + 100)**2 + (df["ball_pos_z"])**2)
df['ball_distance_to_goal_B'] = np.sqrt((df["ball_pos_x"])**2 + (df["ball_pos_y"] - 100)**2 + (df["ball_pos_z"])**2)
    
df['ball_pos'] = np.sqrt(df['ball_pos_x']**2 + df['ball_pos_y']**2 + df['ball_pos_z']**2)
df['ball_vel'] = np.sqrt(df['ball_vel_x']**2 + df['ball_vel_y']**2 + df['ball_vel_z']**2)

df['p0_pos'] = np.sqrt(df['p0_pos_x']**2 + df['p0_pos_y']**2 + df['p0_pos_z']**2)
df['p1_pos'] = np.sqrt(df['p1_pos_x']**2 + df['p1_pos_y']**2 + df['p1_pos_z']**2)
df['p2_pos'] = np.sqrt(df['p2_pos_x']**2 + df['p2_pos_y']**2 + df['p2_pos_z']**2)
df['p3_pos'] = np.sqrt(df['p3_pos_x']**2 + df['p3_pos_y']**2 + df['p3_pos_z']**2)
df['p4_pos'] = np.sqrt(df['p4_pos_x']**2 + df['p4_pos_y']**2 + df['p4_pos_z']**2)
df['p5_pos'] = np.sqrt(df['p5_pos_x']**2 + df['p5_pos_y']**2 + df['p5_pos_z']**2)

df['p0_vel'] = np.sqrt(df['p0_vel_x']**2 + df['p0_vel_y']**2 + df['p0_vel_z']**2)
df['p1_vel'] = np.sqrt(df['p1_vel_x']**2 + df['p1_vel_y']**2 + df['p1_vel_z']**2)
df['p2_vel'] = np.sqrt(df['p2_vel_x']**2 + df['p2_vel_y']**2 + df['p2_vel_z']**2)
df['p3_vel'] = np.sqrt(df['p3_vel_x']**2 + df['p3_vel_y']**2 + df['p3_vel_z']**2)
df['p4_vel'] = np.sqrt(df['p4_vel_x']**2 + df['p4_vel_y']**2 + df['p4_vel_z']**2)
df['p5_vel'] = np.sqrt(df['p5_vel_x']**2 + df['p5_vel_y']**2 + df['p5_vel_z']**2)

df['p0_to_ball'] = np.sqrt((df['p0_pos_x']-df['ball_pos_x'])**2 + (df['p0_pos_y'] - df['ball_pos_y'])**2 + (df['p0_pos_z'] - df['ball_pos_z'])**2)
df['p1_to_ball'] = np.sqrt((df['p1_pos_x']-df['ball_pos_x'])**2 + (df['p1_pos_y'] - df['ball_pos_y'])**2 + (df['p1_pos_z'] - df['ball_pos_z'])**2)
df['p2_to_ball'] = np.sqrt((df['p2_pos_x']-df['ball_pos_x'])**2 + (df['p2_pos_y'] - df['ball_pos_y'])**2 + (df['p2_pos_z'] - df['ball_pos_z'])**2)
df['p3_to_ball'] = np.sqrt((df['p3_pos_x']-df['ball_pos_x'])**2 + (df['p3_pos_y'] - df['ball_pos_y'])**2 + (df['p3_pos_z'] - df['ball_pos_z'])**2)
df['p4_to_ball'] = np.sqrt((df['p4_pos_x']-df['ball_pos_x'])**2 + (df['p4_pos_y'] - df['ball_pos_y'])**2 + (df['p4_pos_z'] - df['ball_pos_z'])**2)
df['p5_to_ball'] = np.sqrt((df['p5_pos_x']-df['ball_pos_x'])**2 + (df['p5_pos_y'] - df['ball_pos_y'])**2 + (df['p5_pos_z'] - df['ball_pos_z'])**2)

df['p0_to_goal'] = np.sqrt((df['p0_pos_x'])**2 + (df['p0_pos_y'] + 100)**2 + (df['p0_pos_z'])**2)
df['p1_to_goal'] = np.sqrt((df['p1_pos_x'])**2 + (df['p1_pos_y'] + 100)**2 + (df['p1_pos_z'])**2)
df['p2_to_goal'] = np.sqrt((df['p2_pos_x'])**2 + (df['p2_pos_y'] + 100)**2 + (df['p2_pos_z'])**2)
df['p3_to_goal'] = np.sqrt((df['p3_pos_x'])**2 + (df['p3_pos_y'] - 100)**2 + (df['p3_pos_z'])**2)
df['p4_to_goal'] = np.sqrt((df['p4_pos_x'])**2 + (df['p4_pos_y'] - 100)**2 + (df['p4_pos_z'])**2)
df['p5_to_goal'] = np.sqrt((df['p5_pos_x'])**2 + (df['p5_pos_y'] - 100)**2 + (df['p5_pos_z'])**2)

In [51]:
df["p0_defense"]  = ((df['p0_pos_x'] > -50) & (df['p0_pos_x'] < 50)) & ((df['p0_pos_y'] > -120) & (df['p0_pos_y'] < -50))
df["p1_defense"]  = ((df['p1_pos_x'] > -50) & (df['p1_pos_x'] < 50)) & ((df['p1_pos_y'] > -120) & (df['p1_pos_y'] < -50))
df["p2_defense"]  = ((df['p2_pos_x'] > -50) & (df['p2_pos_x'] < 50)) & ((df['p2_pos_y'] > -120) & (df['p2_pos_y'] < -50))

df["p3_defense"]  = ((df['p3_pos_x'] > -50) & (df['p3_pos_x'] < 50)) & ((df['p3_pos_y'] < 120) & (df['p3_pos_y'] > 50))
df["p4_defense"]  = ((df['p4_pos_x'] > -50) & (df['p4_pos_x'] < 50)) & ((df['p4_pos_y'] < 120) & (df['p4_pos_y'] > 50))
df["p5_defense"]  = ((df['p5_pos_x'] > -50) & (df['p5_pos_x'] < 50)) & ((df['p5_pos_y'] < 120) & (df['p5_pos_y'] > 50))

In [52]:
# B TAKIMI TARAFI
boost0 = (-61.4, 81.9)
boost1 = (61.4, 81.9)

# ORTA SAHA
boost2 = (-71.7, 0)
boost3 = (71.7, 0)

# A TAKIMI TARAFI
boost4 = (-61.4, -81.9)
boost5 = (61.4, -81.9)

In [53]:
df['p0_to_boost0'] = np.sqrt((df['p0_pos_x'] - boost0[0])**2 + (df['p0_pos_y'] - boost0[1])**2)
df['p0_to_boost1'] = np.sqrt((df['p0_pos_x'] - boost1[0])**2 + (df['p0_pos_y'] - boost1[1])**2)
df['p0_to_boost2'] = np.sqrt((df['p0_pos_x'] - boost2[0])**2 + (df['p0_pos_y'] - boost2[1])**2)
df['p0_to_boost3'] = np.sqrt((df['p0_pos_x'] - boost3[0])**2 + (df['p0_pos_y'] - boost3[1])**2)
df['p0_to_boost4'] = np.sqrt((df['p0_pos_x'] - boost4[0])**2 + (df['p0_pos_y'] - boost4[1])**2)
df['p0_to_boost5'] = np.sqrt((df['p0_pos_x'] - boost5[0])**2 + (df['p0_pos_y'] - boost5[1])**2)

df['p1_to_boost0'] = np.sqrt((df['p1_pos_x'] - boost0[0])**2 + (df['p1_pos_y'] - boost0[1])**2)
df['p1_to_boost1'] = np.sqrt((df['p1_pos_x'] - boost1[0])**2 + (df['p1_pos_y'] - boost1[1])**2)
df['p1_to_boost2'] = np.sqrt((df['p1_pos_x'] - boost2[0])**2 + (df['p1_pos_y'] - boost2[1])**2)
df['p1_to_boost3'] = np.sqrt((df['p1_pos_x'] - boost3[0])**2 + (df['p1_pos_y'] - boost3[1])**2)
df['p1_to_boost4'] = np.sqrt((df['p1_pos_x'] - boost4[0])**2 + (df['p1_pos_y'] - boost4[1])**2)
df['p1_to_boost5'] = np.sqrt((df['p1_pos_x'] - boost5[0])**2 + (df['p1_pos_y'] - boost5[1])**2)

df['p2_to_boost0'] = np.sqrt((df['p2_pos_x'] - boost0[0])**2 + (df['p2_pos_y'] - boost0[1])**2)
df['p2_to_boost1'] = np.sqrt((df['p2_pos_x'] - boost1[0])**2 + (df['p2_pos_y'] - boost1[1])**2)
df['p2_to_boost2'] = np.sqrt((df['p2_pos_x'] - boost2[0])**2 + (df['p2_pos_y'] - boost2[1])**2)
df['p2_to_boost3'] = np.sqrt((df['p2_pos_x'] - boost3[0])**2 + (df['p2_pos_y'] - boost3[1])**2)
df['p2_to_boost4'] = np.sqrt((df['p2_pos_x'] - boost4[0])**2 + (df['p2_pos_y'] - boost4[1])**2)
df['p2_to_boost5'] = np.sqrt((df['p2_pos_x'] - boost5[0])**2 + (df['p2_pos_y'] - boost5[1])**2)

df['p3_to_boost0'] = np.sqrt((df['p3_pos_x'] - boost0[0])**2 + (df['p3_pos_y'] - boost0[1])**2)
df['p3_to_boost1'] = np.sqrt((df['p3_pos_x'] - boost1[0])**2 + (df['p3_pos_y'] - boost1[1])**2)
df['p3_to_boost2'] = np.sqrt((df['p3_pos_x'] - boost2[0])**2 + (df['p3_pos_y'] - boost2[1])**2)
df['p3_to_boost3'] = np.sqrt((df['p3_pos_x'] - boost3[0])**2 + (df['p3_pos_y'] - boost3[1])**2)
df['p3_to_boost4'] = np.sqrt((df['p3_pos_x'] - boost4[0])**2 + (df['p3_pos_y'] - boost4[1])**2)
df['p3_to_boost5'] = np.sqrt((df['p3_pos_x'] - boost5[0])**2 + (df['p3_pos_y'] - boost5[1])**2)

df['p4_to_boost0'] = np.sqrt((df['p4_pos_x'] - boost0[0])**2 + (df['p4_pos_y'] - boost0[1])**2)
df['p4_to_boost1'] = np.sqrt((df['p4_pos_x'] - boost1[0])**2 + (df['p4_pos_y'] - boost1[1])**2)
df['p4_to_boost2'] = np.sqrt((df['p4_pos_x'] - boost2[0])**2 + (df['p4_pos_y'] - boost2[1])**2)
df['p4_to_boost3'] = np.sqrt((df['p4_pos_x'] - boost3[0])**2 + (df['p4_pos_y'] - boost3[1])**2)
df['p4_to_boost4'] = np.sqrt((df['p4_pos_x'] - boost4[0])**2 + (df['p4_pos_y'] - boost4[1])**2)
df['p4_to_boost5'] = np.sqrt((df['p4_pos_x'] - boost5[0])**2 + (df['p4_pos_y'] - boost5[1])**2)

df['p5_to_boost0'] = np.sqrt((df['p5_pos_x'] - boost0[0])**2 + (df['p5_pos_y'] - boost0[1])**2)
df['p5_to_boost1'] = np.sqrt((df['p5_pos_x'] - boost1[0])**2 + (df['p5_pos_y'] - boost1[1])**2)
df['p5_to_boost2'] = np.sqrt((df['p5_pos_x'] - boost2[0])**2 + (df['p5_pos_y'] - boost2[1])**2)
df['p5_to_boost3'] = np.sqrt((df['p5_pos_x'] - boost3[0])**2 + (df['p5_pos_y'] - boost3[1])**2)
df['p5_to_boost4'] = np.sqrt((df['p5_pos_x'] - boost4[0])**2 + (df['p5_pos_y'] - boost4[1])**2)
df['p5_to_boost5'] = np.sqrt((df['p5_pos_x'] - boost5[0])**2 + (df['p5_pos_y'] - boost5[1])**2)

## modelleme

In [54]:
ya = df["team_A_scoring_within_10sec"]
yb = df["team_B_scoring_within_10sec"]
X = df.drop(columns=["team_A_scoring_within_10sec","team_B_scoring_within_10sec"])

Xa_train, Xa_test, ya_train, ya_test = train_test_split(X, ya, test_size=0.30, random_state=22)
Xb_train, Xb_test, yb_train, yb_test = train_test_split(X, yb, test_size=0.30, random_state=22)

In [55]:
params = {"eval_metric" : "Logloss",
          "task_type" : "GPU"}

In [56]:
modela = CatBoostClassifier(**params)
modelb = CatBoostClassifier(**params)

modela.fit(Xa_train,ya_train)
modelb.fit(Xb_train,yb_train)

Learning rate set to 0.022633
0:	learn: 0.6635201	total: 80.5ms	remaining: 1m 20s
1:	learn: 0.6357332	total: 154ms	remaining: 1m 16s
2:	learn: 0.6095061	total: 229ms	remaining: 1m 16s
3:	learn: 0.5848415	total: 284ms	remaining: 1m 10s
4:	learn: 0.5617722	total: 330ms	remaining: 1m 5s
5:	learn: 0.5402031	total: 383ms	remaining: 1m 3s
6:	learn: 0.5200354	total: 431ms	remaining: 1m 1s
7:	learn: 0.5010806	total: 477ms	remaining: 59.1s
8:	learn: 0.4834099	total: 527ms	remaining: 58s
9:	learn: 0.4666829	total: 576ms	remaining: 57s
10:	learn: 0.4511780	total: 615ms	remaining: 55.3s
11:	learn: 0.4364468	total: 666ms	remaining: 54.8s
12:	learn: 0.4226514	total: 715ms	remaining: 54.3s
13:	learn: 0.4098301	total: 762ms	remaining: 53.6s
14:	learn: 0.3977727	total: 805ms	remaining: 52.9s
15:	learn: 0.3865253	total: 856ms	remaining: 52.6s
16:	learn: 0.3758918	total: 904ms	remaining: 52.3s
17:	learn: 0.3659241	total: 946ms	remaining: 51.6s
18:	learn: 0.3565937	total: 989ms	remaining: 51.1s
19:	learn:

In [57]:
ya_pred = modela.predict_proba(Xa_test)
ya_val_pred = modela.predict_proba(Xa_train)
yb_pred = modelb.predict_proba(Xb_test)

In [58]:
preda = pd.DataFrame(data=ya_pred)
valpreda = pd.DataFrame(data=ya_val_pred)
predb = pd.DataFrame(data=yb_pred)

In [59]:
print(f"preda:{log_loss(ya_test, preda[1])}\npredb:{log_loss(yb_test, predb[1])}")
#CPU SKOR 0.13979149870195795
#GPU SKOR 0.19178174905147288
#süreyya nabıyon amk

preda:0.19450641393777782
predb:0.19042976948642676


## test data

In [60]:
test.drop(columns="id", inplace=True)

In [61]:
test_a = test[["p0_pos_x","p1_pos_x","p2_pos_x"]]
test_b = test[["p3_pos_x","p4_pos_x","p5_pos_x"]]

test["team_a_eksik"] = test_a.isnull().sum(axis=1)
test["team_b_eksik"] = test_b.isnull().sum(axis=1)

In [62]:
test['ball_distance_to_goal_A'] = np.sqrt((test["ball_pos_x"])**2 + (test["ball_pos_y"] + 100)**2 + (test["ball_pos_z"])**2)
test['ball_distance_to_goal_B'] = np.sqrt((test["ball_pos_x"])**2 + (test["ball_pos_y"] - 100)**2 + (test["ball_pos_z"])**2)

test['ball_pos'] = np.sqrt(test['ball_pos_x']**2 + test['ball_pos_y']**2 + test['ball_pos_z']**2)
test['ball_vel'] = np.sqrt(test['ball_vel_x']**2 + test['ball_vel_y']**2 + test['ball_vel_z']**2)

test['p0_pos'] = np.sqrt(test['p0_pos_x']**2 + test['p0_pos_y']**2 + test['p0_pos_z']**2)
test['p1_pos'] = np.sqrt(test['p1_pos_x']**2 + test['p1_pos_y']**2 + test['p1_pos_z']**2)
test['p2_pos'] = np.sqrt(test['p2_pos_x']**2 + test['p2_pos_y']**2 + test['p2_pos_z']**2)
test['p3_pos'] = np.sqrt(test['p3_pos_x']**2 + test['p3_pos_y']**2 + test['p3_pos_z']**2)
test['p4_pos'] = np.sqrt(test['p4_pos_x']**2 + test['p4_pos_y']**2 + test['p4_pos_z']**2)
test['p5_pos'] = np.sqrt(test['p5_pos_x']**2 + test['p5_pos_y']**2 + test['p5_pos_z']**2)
    
test['p0_vel'] = np.sqrt(test['p0_vel_x']**2 + test['p0_vel_y']**2 + test['p0_vel_z']**2)
test['p1_vel'] = np.sqrt(test['p1_vel_x']**2 + test['p1_vel_y']**2 + test['p1_vel_z']**2)
test['p2_vel'] = np.sqrt(test['p2_vel_x']**2 + test['p2_vel_y']**2 + test['p2_vel_z']**2)
test['p3_vel'] = np.sqrt(test['p3_vel_x']**2 + test['p3_vel_y']**2 + test['p3_vel_z']**2)
test['p4_vel'] = np.sqrt(test['p4_vel_x']**2 + test['p4_vel_y']**2 + test['p4_vel_z']**2)
test['p5_vel'] = np.sqrt(test['p5_vel_x']**2 + test['p5_vel_y']**2 + test['p5_vel_z']**2)
    
test['p0_to_ball'] = np.sqrt((test['p0_pos_x'] - test['ball_pos_x'])**2 + (test['p0_pos_y'] - test['ball_pos_y'])**2 + (test['p0_pos_z'] - test['ball_pos_z'])**2)
test['p1_to_ball'] = np.sqrt((test['p1_pos_x'] - test['ball_pos_x'])**2 + (test['p1_pos_y'] - test['ball_pos_y'])**2 + (test['p1_pos_z'] - test['ball_pos_z'])**2)
test['p2_to_ball'] = np.sqrt((test['p2_pos_x'] - test['ball_pos_x'])**2 + (test['p2_pos_y'] - test['ball_pos_y'])**2 + (test['p2_pos_z'] - test['ball_pos_z'])**2)
test['p3_to_ball'] = np.sqrt((test['p3_pos_x'] - test['ball_pos_x'])**2 + (test['p3_pos_y'] - test['ball_pos_y'])**2 + (test['p3_pos_z'] - test['ball_pos_z'])**2)
test['p4_to_ball'] = np.sqrt((test['p4_pos_x'] - test['ball_pos_x'])**2 + (test['p4_pos_y'] - test['ball_pos_y'])**2 + (test['p4_pos_z'] - test['ball_pos_z'])**2)
test['p5_to_ball'] = np.sqrt((test['p5_pos_x'] - test['ball_pos_x'])**2 + (test['p5_pos_y'] - test['ball_pos_y'])**2 + (test['p5_pos_z'] - test['ball_pos_z'])**2)

test['p0_to_goal'] = np.sqrt((test['p0_pos_x'])**2 + (test['p0_pos_y'] + 100)**2 + (test['p0_pos_z'])**2)
test['p1_to_goal'] = np.sqrt((test['p1_pos_x'])**2 + (test['p1_pos_y'] + 100)**2 + (test['p1_pos_z'])**2)
test['p2_to_goal'] = np.sqrt((test['p2_pos_x'])**2 + (test['p2_pos_y'] + 100)**2 + (test['p2_pos_z'])**2)
test['p3_to_goal'] = np.sqrt((test['p3_pos_x'])**2 + (test['p3_pos_y'] - 100)**2 + (test['p3_pos_z'])**2)
test['p4_to_goal'] = np.sqrt((test['p4_pos_x'])**2 + (test['p4_pos_y'] - 100)**2 + (test['p4_pos_z'])**2)
test['p5_to_goal'] = np.sqrt((test['p5_pos_x'])**2 + (test['p5_pos_y'] - 100)**2 + (test['p5_pos_z'])**2)

In [63]:
test["p0_defense"]  = ((test['p0_pos_x'] > -50) & (test['p0_pos_x'] < 50)) & ((test['p0_pos_y'] > -120) & (test['p0_pos_y'] < -50))
test["p1_defense"]  = ((test['p1_pos_x'] > -50) & (test['p1_pos_x'] < 50)) & ((test['p1_pos_y'] > -120) & (test['p1_pos_y'] < -50))
test["p2_defense"]  = ((test['p2_pos_x'] > -50) & (test['p2_pos_x'] < 50)) & ((test['p2_pos_y'] > -120) & (test['p2_pos_y'] < -50))

test["p3_defense"]  = ((test['p3_pos_x'] > -50) & (test['p3_pos_x'] < 50)) & ((test['p3_pos_y'] < 120) & (test['p3_pos_y'] > 50))
test["p4_defense"]  = ((test['p4_pos_x'] > -50) & (test['p4_pos_x'] < 50)) & ((test['p4_pos_y'] < 120) & (test['p4_pos_y'] > 50))
test["p5_defense"]  = ((test['p5_pos_x'] > -50) & (test['p5_pos_x'] < 50)) & ((test['p5_pos_y'] < 120) & (test['p5_pos_y'] > 50))

In [64]:
test['p0_to_boost0'] = np.sqrt((test['p0_pos_x'] - boost0[0])**2 + (test['p0_pos_y'] - boost0[1])**2)
test['p0_to_boost1'] = np.sqrt((test['p0_pos_x'] - boost1[0])**2 + (test['p0_pos_y'] - boost1[1])**2)
test['p0_to_boost2'] = np.sqrt((test['p0_pos_x'] - boost2[0])**2 + (test['p0_pos_y'] - boost2[1])**2)
test['p0_to_boost3'] = np.sqrt((test['p0_pos_x'] - boost3[0])**2 + (test['p0_pos_y'] - boost3[1])**2)
test['p0_to_boost4'] = np.sqrt((test['p0_pos_x'] - boost4[0])**2 + (test['p0_pos_y'] - boost4[1])**2)
test['p0_to_boost5'] = np.sqrt((test['p0_pos_x'] - boost5[0])**2 + (test['p0_pos_y'] - boost5[1])**2)

test['p1_to_boost0'] = np.sqrt((test['p1_pos_x'] - boost0[0])**2 + (test['p1_pos_y'] - boost0[1])**2)
test['p1_to_boost1'] = np.sqrt((test['p1_pos_x'] - boost1[0])**2 + (test['p1_pos_y'] - boost1[1])**2)
test['p1_to_boost2'] = np.sqrt((test['p1_pos_x'] - boost2[0])**2 + (test['p1_pos_y'] - boost2[1])**2)
test['p1_to_boost3'] = np.sqrt((test['p1_pos_x'] - boost3[0])**2 + (test['p1_pos_y'] - boost3[1])**2)
test['p1_to_boost4'] = np.sqrt((test['p1_pos_x'] - boost4[0])**2 + (test['p1_pos_y'] - boost4[1])**2)
test['p1_to_boost5'] = np.sqrt((test['p1_pos_x'] - boost5[0])**2 + (test['p1_pos_y'] - boost5[1])**2)

test['p2_to_boost0'] = np.sqrt((test['p2_pos_x'] - boost0[0])**2 + (test['p2_pos_y'] - boost0[1])**2)
test['p2_to_boost1'] = np.sqrt((test['p2_pos_x'] - boost1[0])**2 + (test['p2_pos_y'] - boost1[1])**2)
test['p2_to_boost2'] = np.sqrt((test['p2_pos_x'] - boost2[0])**2 + (test['p2_pos_y'] - boost2[1])**2)
test['p2_to_boost3'] = np.sqrt((test['p2_pos_x'] - boost3[0])**2 + (test['p2_pos_y'] - boost3[1])**2)
test['p2_to_boost4'] = np.sqrt((test['p2_pos_x'] - boost4[0])**2 + (test['p2_pos_y'] - boost4[1])**2)
test['p2_to_boost5'] = np.sqrt((test['p2_pos_x'] - boost5[0])**2 + (test['p2_pos_y'] - boost5[1])**2)

test['p3_to_boost0'] = np.sqrt((test['p3_pos_x'] - boost0[0])**2 + (test['p3_pos_y'] - boost0[1])**2)
test['p3_to_boost1'] = np.sqrt((test['p3_pos_x'] - boost1[0])**2 + (test['p3_pos_y'] - boost1[1])**2)
test['p3_to_boost2'] = np.sqrt((test['p3_pos_x'] - boost2[0])**2 + (test['p3_pos_y'] - boost2[1])**2)
test['p3_to_boost3'] = np.sqrt((test['p3_pos_x'] - boost3[0])**2 + (test['p3_pos_y'] - boost3[1])**2)
test['p3_to_boost4'] = np.sqrt((test['p3_pos_x'] - boost4[0])**2 + (test['p3_pos_y'] - boost4[1])**2)
test['p3_to_boost5'] = np.sqrt((test['p3_pos_x'] - boost5[0])**2 + (test['p3_pos_y'] - boost5[1])**2)

test['p4_to_boost0'] = np.sqrt((test['p4_pos_x'] - boost0[0])**2 + (test['p4_pos_y'] - boost0[1])**2)
test['p4_to_boost1'] = np.sqrt((test['p4_pos_x'] - boost1[0])**2 + (test['p4_pos_y'] - boost1[1])**2)
test['p4_to_boost2'] = np.sqrt((test['p4_pos_x'] - boost2[0])**2 + (test['p4_pos_y'] - boost2[1])**2)
test['p4_to_boost3'] = np.sqrt((test['p4_pos_x'] - boost3[0])**2 + (test['p4_pos_y'] - boost3[1])**2)
test['p4_to_boost4'] = np.sqrt((test['p4_pos_x'] - boost4[0])**2 + (test['p4_pos_y'] - boost4[1])**2)
test['p4_to_boost5'] = np.sqrt((test['p4_pos_x'] - boost5[0])**2 + (test['p4_pos_y'] - boost5[1])**2)

test['p5_to_boost0'] = np.sqrt((test['p5_pos_x'] - boost0[0])**2 + (test['p5_pos_y'] - boost0[1])**2)
test['p5_to_boost1'] = np.sqrt((test['p5_pos_x'] - boost1[0])**2 + (test['p5_pos_y'] - boost1[1])**2)
test['p5_to_boost2'] = np.sqrt((test['p5_pos_x'] - boost2[0])**2 + (test['p5_pos_y'] - boost2[1])**2)
test['p5_to_boost3'] = np.sqrt((test['p5_pos_x'] - boost3[0])**2 + (test['p5_pos_y'] - boost3[1])**2)
test['p5_to_boost4'] = np.sqrt((test['p5_pos_x'] - boost4[0])**2 + (test['p5_pos_y'] - boost4[1])**2)
test['p5_to_boost5'] = np.sqrt((test['p5_pos_x'] - boost5[0])**2 + (test['p5_pos_y'] - boost5[1])**2)

In [65]:
test_preda = pd.DataFrame(data=modela.predict_proba(test))
test_predb = pd.DataFrame(data=modelb.predict_proba(test))

In [66]:
sub["team_A_scoring_within_10sec"] = test_preda[1]
sub["team_B_scoring_within_10sec"] = test_predb[1]

In [67]:
sub.to_csv("submission.csv",index=False)

In [68]:
sub

,id,team_A_scoring_within_10sec,team_B_scoring_within_10sec
0,0,0.029957,0.030975
1,1,0.027461,0.071945
2,2,0.018099,0.089003
3,3,0.048964,0.029198
4,4,0.021068,0.059700
...,...,...,...
701138,701138,0.361076,0.037133
701139,701139,0.019704,0.057454
701140,701140,0.092463,0.039448
701141,701141,0.016324,0.135547
